# RNN Basics
In this notebook, we will first implement a standard RNN, run it step by step, and observe the vanishing gradients problem. Then we learn how to use Pytorch's RNN implementations. 

In [1]:
# first, we create the vectors for the input vector
import torch
import torch.nn as nn

token_num = 10
embd_dim = 5

input_matrix = torch.randn(token_num, embd_dim) # each row corresponds to a token's vector
print(input_matrix)
print(input_matrix.shape)

tensor([[ 2.0822,  0.1974, -0.0715, -1.4178, -1.3477],
        [ 2.0813,  1.0990,  1.6563, -1.0923, -2.1113],
        [ 1.0184, -1.0791,  1.8385, -0.7616,  0.1962],
        [-0.7798, -0.3810,  1.5821,  0.3865,  0.9165],
        [-1.3580,  0.4285,  0.0228,  1.0516,  0.7123],
        [-1.3944,  1.3972,  0.5477,  1.0267,  1.0476],
        [-0.1006,  0.1568,  1.0043, -0.1115, -1.7338],
        [-1.0767, -0.9785,  0.7733, -0.3949, -0.4754],
        [ 2.1951,  0.9607, -0.4603, -0.2557, -0.2927],
        [-0.5441,  0.7239,  0.3841,  0.4805, -0.9819]])
torch.Size([10, 5])


In [2]:
# then we define the RNN architecture
class Basic_RNN(nn.Module):
    def __init__(self, embd_dim, hidden_dim):
        super(Basic_RNN, self).__init__()
        self.fc = nn.Linear(embd_dim+hidden_dim, hidden_dim) # maps the concatenation of input vec and previous hidden vec to the new hidden vec
        self.init_hidden = torch.randn(hidden_dim) # h_0: hidden state 0
        self.init_hidden.requires_grad_(True) # we will compute the gradient wrt. h_0 later
        self.act_fnc = nn.ReLU()
    def forward(self, input_vec, hidden_vec):
        cat_vec = torch.cat( (input_vec, hidden_vec) )
        return self.act_fnc( self.fc(cat_vec) )   

In [3]:
# then we initialize the RNN defined above
hidden_dim = 7
rnn = Basic_RNN(embd_dim, hidden_dim)
for pname, para in rnn.named_parameters():
    print(pname, para.shape)

fc.weight torch.Size([7, 12])
fc.bias torch.Size([7])


If we ignore the biases, the size of parameters to learn is (embd_dim+hidden_dim)\*hidden_dim, i.e. $7\times12$ in our example. We later will compare it with the pytorch implementation's parameter size.

In [4]:
# now we pass the input matrix into RNN, and compute the hidden state at each step
hidden_vecs = []
hidden_vecs.append(rnn.init_hidden) # first add h_0 to the list

# iterate over all input tokens
for i in range(token_num):
    word_vec = input_matrix[i] # get the input token's vector
    new_hidden_state = rnn(word_vec, hidden_vecs[-1]) # get new hidden state
    hidden_vecs.append(new_hidden_state)
    
print('total hidden state num: ',len(hidden_vecs))

total hidden state num:  11


## Vanishing gradients
Below we study the gradient of the last hidden state wrt. the first hidden state, to get a better idea of how the vanishing gradient problem looks like. 

In [5]:
def check_gradient(tnum):
    temp_input_matrix = torch.randn(tnum, embd_dim) 
    temp_rnn = Basic_RNN(embd_dim, hidden_dim)
    temp_hidden_vecs = []
    temp_hidden_vecs.append(temp_rnn.init_hidden)
    for i in range(tnum):
        word_vec = temp_input_matrix[i]
        new_hidden_state = temp_rnn(word_vec, temp_hidden_vecs[-1])
        temp_hidden_vecs.append(new_hidden_state)
    
    hv = temp_hidden_vecs[-1]
    loss = torch.dot(hv, hv) # L2 norm of the 
    loss.backward(retain_graph=True)
    print('For an input sentence with {} tokens, the gradient of the last hidden state wrt. the first hidden state is: \n{}'.format(
        tnum, temp_hidden_vecs[0].grad))
    
check_gradient(tnum=20) # try tnum=2, 20, 200, run each value for multiple times and observe the gradients

For an input sentence with 20 tokens, the gradient of the last hidden state wrt. the first hidden state is: 
tensor([ 5.3330e-10, -2.2074e-10,  5.3987e-10, -5.8066e-10, -5.0050e-10,
         6.9781e-10,  7.2531e-10])


Try different tnum values and observe the gradients. Generally you will find that with larger tnum values, the gradient values become smaller, reflecting the vanishing gradient problem. 

## Use Pytorch's RNN implementation
Pytorch.nn provides implementations for three RNN architectures: standard RNN, LSTM ans GRU. Below we look at how to use the standard RNN implementation from Pytorch.

In [6]:
# we initialize a vanilla RNN
vrnn = nn.RNN(hidden_size = hidden_dim, batch_first=True, input_size=embd_dim, nonlinearity='tanh')
print(vrnn)

RNN(5, 7, batch_first=True)


In [7]:
# before we pass the input matrix into the RNN, we first take a look at its parameters
for name, para in vrnn.named_parameters():
    print('\n======', name)
    print(para.shape)
    print(para)


====== weight_ih_l0
torch.Size([7, 5])
Parameter containing:
tensor([[-0.2237, -0.1901,  0.2603, -0.3154, -0.0487],
        [-0.2849, -0.0017,  0.1238, -0.0078, -0.3110],
        [-0.0893,  0.2164,  0.3321, -0.3331, -0.3623],
        [ 0.1474,  0.3701, -0.1808, -0.2338, -0.0891],
        [-0.1137, -0.0864, -0.2359,  0.0590, -0.3041],
        [ 0.3007, -0.3160,  0.3409,  0.1242,  0.2560],
        [ 0.0661, -0.3052, -0.1826,  0.3742, -0.3736]], requires_grad=True)

====== weight_hh_l0
torch.Size([7, 7])
Parameter containing:
tensor([[ 0.1494,  0.1714, -0.3257,  0.0428, -0.3640, -0.0078,  0.3209],
        [-0.0019,  0.2766,  0.0590,  0.1654,  0.2182,  0.0052,  0.1342],
        [ 0.3729, -0.2813,  0.2539,  0.1346,  0.1792,  0.2410, -0.1724],
        [ 0.0401, -0.1273, -0.0700, -0.2556,  0.3589, -0.2135, -0.0862],
        [ 0.2470, -0.3395,  0.3354, -0.0076,  0.1569, -0.0640,  0.0626],
        [ 0.1253,  0.1516,  0.1562, -0.2048, -0.2891,  0.3160, -0.0869],
        [-0.0858,  0.0286,  0.24

We can see that in Pytorch's implementation, a vanilla RNN includes four parts of parameters, which jointly constitute the parameter matrix **$W$** introduced in our lecture slides.

* weight_ih_l0: linearly maps the input vectors to a vector of hidden_dim; let's denote its resulting vector $h_i$. In our example, the dimension of this parameter matrix should be $5\times7$; in the implementation, it is transposed to ease computation. *ih* stands for input to hidden, *l0* stands for layer 0 (the first layer).
* weight_hh_l0: linearly maps the previous hidden vector to a vector of hidden_dim; let's denote the resulting vector as $h_h$. Hence its dimension is $7\times7$ in our example. 
* bias_ih_l0 and bias_hh_l0: the bias vectors used to perform the linear mapping. To ease your understanding, we do not explicitly include them in the RNN diagrams in the lecture slides. 

The new hidden vector will be the sum of $h_i$, $h_h$ and the two biases. If we ignore the 
biases, its total parameter size is (embd_dim+hidden_dim)\*hidden_dim, same as our own implementation's. 

In [8]:
# then we pass the input to the RNN, and get the hidden vectors at each step
input_matrix = input_matrix.unsqueeze(0)
output_hidden_vecs = vrnn(input_matrix)[0] 
print(output_hidden_vecs.shape)
print(output_hidden_vecs)

torch.Size([1, 10, 7])
tensor([[[-0.5657,  0.4513,  0.7584,  0.6454, -0.3040, -0.1373, -0.2110],
         [-0.5446,  0.8062,  0.9488,  0.3975, -0.5673,  0.0572, -0.4303],
         [-0.1122,  0.6090,  0.4765, -0.7484, -0.8264,  0.8882, -0.5307],
         [-0.1152,  0.5478,  0.2120, -0.8196, -0.8335,  0.8903, -0.5082],
         [-0.5811,  0.5377, -0.2808, -0.6394, -0.6654,  0.4274, -0.2706],
         [-0.5344,  0.5618, -0.3873, -0.3912, -0.8660,  0.0121, -0.7102],
         [-0.0451,  0.8130,  0.6060, -0.2475, -0.6365,  0.0022, -0.1200],
         [ 0.2559,  0.8561,  0.4020, -0.7192, -0.4194,  0.3628, -0.0826],
         [-0.8287,  0.1075,  0.1348,  0.5422, -0.6259,  0.5017, -0.1044],
         [-0.5279,  0.8152,  0.4794, -0.4631, -0.4656, -0.4123, -0.2593]]],
       grad_fn=<TransposeBackward1>)


We can see that the (first) output of vrnn is the hidden vectors of each step. We may either use the last hidden state as the representation of the input text, or perform average or max pooling over all hidden vectors.

In [9]:
# suppose we want to use the average hidden vector as the input representation
max_pooler = nn.MaxPool1d(token_num)
input_repr = max_pooler(torch.transpose(output_hidden_vecs,1,2))
print(input_repr)

tensor([[[ 0.2559],
         [ 0.8561],
         [ 0.9488],
         [ 0.6454],
         [-0.3040],
         [ 0.8903],
         [-0.0826]]], grad_fn=<SqueezeBackward1>)


In [10]:
# or we can use the average hidden vector as the input representation
avg_pooler = nn.AvgPool1d(token_num)
input_repr = avg_pooler(torch.transpose(output_hidden_vecs,1,2))
print(input_repr)

tensor([[[-0.3599],
         [ 0.6106],
         [ 0.3350],
         [-0.2443],
         [-0.6210],
         [ 0.2592],
         [-0.3227]]], grad_fn=<SqueezeBackward1>)


LSTM and GRU implementations can be used in a similar manner.

In [11]:
# uni-direction lstm
lstm = nn.LSTM(hidden_size = hidden_dim, batch_first=True, input_size=embd_dim)
lstm_hidden_states = lstm(input_matrix)[0]
print(lstm_hidden_states.shape)

torch.Size([1, 10, 7])


In [12]:
# bi-direction lstm
bilstm = nn.LSTM(hidden_size = hidden_dim, batch_first=True, input_size=embd_dim, bidirectional=True)
bilstm_hidden_states = bilstm(input_matrix)[0]
print(bilstm_hidden_states.shape)

torch.Size([1, 10, 14])


In [13]:
# GRU
gru = nn.GRU(hidden_size = hidden_dim, batch_first=True, input_size=embd_dim)
gru_hidden_states = gru(input_matrix)[0]
print(gru_hidden_states.shape)

torch.Size([1, 10, 7])


In [21]:
# perform avg pooling over the hidden vectors
import numpy as np
print(lstm_hidden_states)
avg_pooler = nn.AvgPool1d(token_num)
input_repr = avg_pooler(torch.transpose(lstm_hidden_states,1,2))
print(input_repr.squeeze())

tensor([[[ 0.1977,  0.2796,  0.0602,  0.3116,  0.0238,  0.1606,  0.0139],
         [ 0.3045,  0.4148,  0.1031,  0.4776, -0.0724,  0.0485, -0.0582],
         [ 0.2838,  0.5039,  0.2338,  0.5297, -0.1518,  0.0448, -0.0689],
         [ 0.2526,  0.3580,  0.2289,  0.3288, -0.2254, -0.0194, -0.1884],
         [ 0.2973,  0.2431,  0.1304,  0.1231, -0.2249,  0.0112, -0.2876],
         [ 0.3203,  0.1745,  0.1305, -0.0125, -0.2363, -0.0786, -0.3694],
         [ 0.3301,  0.2842,  0.2212,  0.1933, -0.0888, -0.1600, -0.2467],
         [ 0.1720,  0.3268,  0.3069,  0.2408,  0.0543, -0.0795, -0.0795],
         [ 0.4347,  0.3756,  0.1447,  0.3704, -0.0587,  0.2335, -0.1874],
         [ 0.3736,  0.2752,  0.1236,  0.3348, -0.0890,  0.0310, -0.2200]]],
       grad_fn=<TransposeBackward0>)
tensor([ 0.2967,  0.3236,  0.1683,  0.2898, -0.1069,  0.0192, -0.1692],
       grad_fn=<SqueezeBackward0>)
